In [8]:
# from huggingface_hub import snapshot_download

# snapshot_download(
#     repo_id="Fhrozen/FSD50k",
#     repo_type="dataset",
#     local_dir = './FSD50k', max_workers = 30,
# )
# !huggingface-cli download Fhrozen/FSD50k --local-dir="./FSD50k" --max-workers="30" --repo-type=dataset

In [47]:
import librosa
import soundfile as sf
import pandas as pd
import os
import json
from glob import glob
from tqdm import tqdm
from multiprocess import Pool
import itertools
import soundfile as sf

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

In [44]:
!mkdir FSD50k-audio

mkdir: cannot create directory ‘FSD50k-audio’: File exists


In [49]:
def loop(files):
    files, _ = files
    for f in tqdm(files):
        y, sr = librosa.load(f, sr = 16000)
        new_f = os.path.join('FSD50k-audio', f.replace('/', '_').replace('.wav', '.mp3'))
        sf.write(new_f, y, sr)

In [43]:
audio_files = glob('FSD50k/clips/*/*.wav')
len(audio_files)

51197

In [52]:
# multiprocessing(audio_files, loop, cores = 20)

In [67]:
data = []

df = pd.read_csv('FSD50k/labels/dev.csv')
for i in tqdm(range(len(df))):
    audio_filename = os.path.join(f"FSD50k/clips/dev/{df['fname'].iloc[i]}.wav")
    new_f = os.path.join('FSD50k-audio', audio_filename.replace('/', '_').replace('.wav', '.mp3'))
    if not os.path.exists(new_f):
        continue
    data.append({
        'answer': df['labels'].iloc[i],
        'audio_filename': new_f,
        'metadata': json.dumps(df.iloc[i].to_dict())
    })

100%|██████████████████████████████████████████████████████████████████████████████| 40966/40966 [00:02<00:00, 15586.31it/s]


In [68]:
df = pd.read_csv('FSD50k/labels/eval.csv')
for i in tqdm(range(len(df))):
    audio_filename = os.path.join(f"FSD50k/clips/eval/{df['fname'].iloc[i]}.wav")
    new_f = os.path.join('FSD50k-audio', audio_filename.replace('/', '_').replace('.wav', '.mp3'))
    if not os.path.exists(new_f):
        continue
    data.append({
        'answer': df['labels'].iloc[i],
        'audio_filename': new_f,
        'metadata': json.dumps(df.iloc[i].to_dict())
    })

100%|██████████████████████████████████████████████████████████████████████████████| 10231/10231 [00:00<00:00, 15799.14it/s]


In [69]:
len(data)

51197

In [70]:
labels = []
for d in data:
    labels.extend(d['answer'].split(','))
    
labels = str(list(set(labels)))

In [71]:
import random

questions = [
    'given the labels\n{labels}\n\nclassify the audio multilabels',
    'what are the multilabels for audio\n\nthe labels: {labels}'
]

for i in range(len(data)):
    data[i]['question'] = random.choice(questions).format(labels = labels)

In [72]:
data[0]

{'answer': 'Electric_guitar,Guitar,Plucked_string_instrument,Musical_instrument,Music',
 'audio_filename': 'FSD50k-audio/FSD50k_clips_dev_64760.mp3',
 'metadata': '{"fname": 64760, "labels": "Electric_guitar,Guitar,Plucked_string_instrument,Musical_instrument,Music", "mids": "/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf", "split": "train"}',
 'question': "given the labels\n['Crying_and_sobbing', 'Cat', 'Raindrop', 'Gull_and_seagull', 'Hiss', 'Truck', 'Human_voice', 'Cheering', 'Speech_synthesizer', 'Cupboard_open_or_close', 'Speech', 'Wild_animals', 'Thunderstorm', 'Child_speech_and_kid_speaking', 'Typewriter', 'Train', 'Cymbal', 'Crackle', 'Plucked_string_instrument', 'Bicycle_bell', 'Cricket', 'Crushing', 'Splash_and_splatter', 'Stream', 'Fart', 'Mechanical_fan', 'Bicycle', 'Fireworks', 'Screech', 'Traffic_noise_and_roadway_noise', 'Race_car_and_auto_racing', 'Marimba_and_xylophone', 'Drip', 'Wind_instrument_and_woodwind_instrument', 'Burping_and_eructation', 'Acoustic_guitar', 'Com

In [73]:
from datasets import Dataset

dataset = Dataset.from_list(data)

In [74]:
dataset.push_to_hub('mesolitica/Zeroshot-Audio-Classification-Instructions', split = 'fsd50k')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/52 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Zeroshot-Audio-Classification-Instructions/commit/3ca4cf62acbf9fa9940bbba8e65dee33b290b55e', commit_message='Upload dataset', commit_description='', oid='3ca4cf62acbf9fa9940bbba8e65dee33b290b55e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Zeroshot-Audio-Classification-Instructions', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Zeroshot-Audio-Classification-Instructions'), pr_revision=None, pr_num=None)

In [75]:
from glob import glob

audio_files = glob('FSD50k-audio/*.mp3')
len(audio_files)

51197

In [76]:
!du -hs FSD50k-audio

1.8G	FSD50k-audio


In [77]:
import zipfile

with zipfile.ZipFile('FSD50k-audio.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    for f in audio_files:
        zipf.write(f, arcname=f)

In [78]:
!huggingface-cli upload mesolitica/Zeroshot-Audio-Classification-Instructions FSD50k-audio.zip \
--repo-type=dataset

Uploading files using Xet Storage..
Uploading...: 100%|████████████████████████| 1.77G/1.77G [00:33<00:00, 53.2MB/s]
https://huggingface.co/datasets/mesolitica/Zeroshot-Audio-Classification-Instructions/blob/main/FSD50k-audio.zip


In [79]:
!rm -rf FSD50k FSD50k-audio.zip